<a href="https://colab.research.google.com/github/dangoldmann/TP-Netflix/blob/main/TP_Netflix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Trabajo Práctico Netflix**
### *Septiembre 2022*

Descargar el archivo que contiene el historial de Netflix. Pueden descargar el suyo propio entrando a este link. A resolver de a grupos de 2 o 3 integrantes.

Ignorar los siguientes dispositivos: 'Sony PS4', 'Netflix Windows App - Cadmium Windows Mobile', 'Apple iPhone 6s', 'Apple iPhone 7 Plus (CDMA)', 'Chrome PC (Cadmium)', 'Samsung 2015 NT14M DTV Smart TV', 'iPhone 5S GSM', 'Google Chromecast V3 Streaming Stick'.

- Contabilizar el dispositivo con el que más títulos se reprodujeron.
- Encontrar la película con más ocurrencias y la película con mayor duración total.
- Generar un TOP “N” de series vistas y graficar.
 - Por tiempo visto
 - Por ocurrencias
- Visualizar una semana de reproducciones genérica, en función a las reproducciones por día y por horario de todos los años.
Contabilizar la cantidad de títulos vistos por año, graficar su variación a través de los años.
Total de días de todos los títulos reproducidos.

### **Bonus**

- Identificar cuál fue la película de Star Wars más veces vista por duración.
- Utilizando la API de IMDb (u otra):
 - Mostrar el póster de las películas más vistas, y de las series.
 - Encontrar el género de series y el de películas más ocurrente.
- Encontrar la serie que más rápido se terminó de ver.


Formato de entrega: Link a GitHub conteniendo el Ipython Notebook con su output guardado. No se corrigen Notebooks sin output generado ni entregas que no sean links a GitHub directamente. Se va a evaluar formato del Notebook, celdas de texto, prolijidad del código y de los resultados generados.




In [2]:
import numpy as np
import pandas as pd

Guardando el dataframe

In [152]:
df = pd.read_csv("NetflixHistory.csv")

Eliminando del dataframe las visualizaciones de los dispositivos no deseados

In [174]:
dispositivosIgnorados = ["Sony PS4", "Netflix Windows App - Cadmium Windows Mobile", "Apple iPhone 6s", "Apple iPhone 7 Plus (CDMA)", "Chrome PC (Cadmium)", "Samsung 2015 NT14M DTV Smart TV", "iPhone 5S GSM", "Google Chromecast V3 Streaming Stick"]

for i in dispositivosIgnorados:
  df = df[df["Device Type"] != i]

Contabilizar el dispositivo con el que más títulos se reprodujeron

In [112]:
dispositivos = df.groupby(["Device Type"]).size()
max_cant = max(dispositivos)
dispositivos = dispositivos.reset_index()

In [113]:
dispositivoMasUtilizado = dispositivos.loc[dispositivos[0]==max_cant]
dispositivoMasUtilizado

,Device Type,0
1,Apple Apple TV 4 Apple TV,3519


Crear una nueva columna que contenga si la visualización corresponde a una película o una serie

In [173]:
def TitleType(input):
  if "hook" or "Trailer" in input:
    return
  if "Temporada" in input:
    return "Serie"
  else:
    return "Pelicula"

In [176]:
df["Type"] = df["Title"].apply(TitleType)

Encontrar la película con más ocurrencias y la película con mayor duración total.

In [175]:
df

,Unnamed: 0,Profile Name,Start Time,Duration,Attributes,Title,Supplemental Video Type,Device Type,Bookmark,Latest Bookmark,Country,Type
0,0,Chona,2022-08-21 19:54:53,00:02:54,NaN,Community: Temporada 3: Teoría del caos correc...,NaN,Apple Apple TV 4 Apple TV,00:02:55,00:02:55,AR (Argentina),Serie
1,1,Chona,2022-08-21 19:33:17,00:21:33,NaN,Community: Temporada 3: Ecología competitiva (...,NaN,Apple Apple TV 4 Apple TV,00:21:33,00:21:33,AR (Argentina),Serie
2,2,Chona,2022-08-21 19:11:40,00:21:28,NaN,Community: Temporada 3: Geografía del conflict...,NaN,Apple Apple TV 4 Apple TV,00:21:28,00:21:28,AR (Argentina),Serie
3,3,Chona,2022-08-21 18:50:27,00:20:41,NaN,Community: Temporada 2: Intermediate Documenta...,NaN,Apple Apple TV 4 Apple TV,00:21:02,00:21:02,AR (Argentina),Serie
4,4,Chona,2022-08-21 18:16:34,00:21:20,NaN,Community: Temporada 3: El documental (Capítul...,NaN,Apple Apple TV 4 Apple TV,00:21:21,00:21:21,AR (Argentina),Serie
...,...,...,...,...,...,...,...,...,...,...,...,...
12992,12992,Chona,2015-01-08 19:21:21,00:04:09,NaN,Lost: Temporada 1: Éxodo: Parte 1 (Capítulo 23),NaN,Apple TV 3 (revised),00:04:40,Not latest view,AR (Argentina),Serie
12993,12993,Chona,2015-01-08 19:20:47,00:00:15,NaN,Lost: Temporada 1: Nacida para correr (Capítul...,NaN,Apple TV 3 (revised),00:00:14,Not latest view,AR (Argentina),Serie
12994,12994,Chona,2015-01-08 19:18:32,00:00:37,NaN,Lost: Temporada 1: Éxodo: Parte 1 (Capítulo 23),NaN,Apple TV 3 (revised),00:00:37,Not latest view,AR (Argentina),Serie
12995,12995,Chona,2015-01-08 19:16:45,00:00:18,NaN,Lost: Temporada 1: Éxodo: Parte 1 (Capítulo 23),NaN,Apple TV 3 (revised),00:00:18,Not latest view,AR (Argentina),Serie
